# Dependencies

In [1]:
import os

import imageio
import gymnasium as gym
import numpy as np
import torch
from agilerl.algorithms.ppo import PPO
from agilerl.hpo.mutation import Mutations
from agilerl.hpo.tournament import TournamentSelection
from agilerl.training.train_on_policy import train_on_policy
from agilerl.utils.utils import create_population, make_vect_envs
from tqdm import trange

# Defining Hyperparameters

In [2]:
# Initial hyperparameters
INIT_HP = {
    "POP_SIZE": 4,  # Population size
    "DISCRETE_ACTIONS": True,  # Discrete action space
    "BATCH_SIZE": 128,  # Batch size
    "LR": 0.001,  # Learning rate
    "LEARN_STEP": 1024,  # Learning frequency
    "GAMMA": 0.99,  # Discount factor
    "GAE_LAMBDA": 0.95,  # Lambda for general advantage estimation
    "ACTION_STD_INIT": 0.6,  # Initial action standard deviation
    "CLIP_COEF": 0.2,  # Surrogate clipping coefficient
    "ENT_COEF": 0.01,  # Entropy coefficient
    "VF_COEF": 0.5,  # Value function coefficient
    "MAX_GRAD_NORM": 0.5,  # Maximum norm for gradient clipping
    "TARGET_KL": None,  # Target KL divergence threshold
    "UPDATE_EPOCHS": 4,  # Number of policy update epochs
    # Swap image channels dimension from last to first [H, W, C] -> [C, H, W]
    "CHANNELS_LAST": False,  # Use with RGB states
    "TARGET_SCORE": 200.0,  # Target score that will beat the environment
    "MAX_STEPS": 150000,  # Maximum number of steps an agent takes in an environment
    "EVO_STEPS": 10000,  # Evolution frequency
    "EVAL_STEPS": None,  # Number of evaluation steps per episode
    "EVAL_LOOP": 3,  # Number of evaluation episodes
    "TOURN_SIZE": 2,  # Tournament size
    "ELITISM": True,  # Elitism in tournament selection
}

# Mutation parameters
MUT_P = {
    # Mutation probabilities
    "NO_MUT": 0.4,  # No mutation
    "ARCH_MUT": 0.2,  # Architecture mutation
    "NEW_LAYER": 0.2,  # New layer mutation
    "PARAMS_MUT": 0.2,  # Network parameters mutation
    "ACT_MUT": 0.2,  # Activation layer mutation
    "RL_HP_MUT": 0.2,  # Learning HP mutation
    # Learning HPs to choose from
    "RL_HP_SELECTION": ["lr", "batch_size", "learn_step"],
    "MUT_SD": 0.1,  # Mutation strength
    "RAND_SEED": 42,  # Random seed
    # Define max and min limits for mutating RL hyperparams
    "MIN_LR": 0.0001,
    "MAX_LR": 0.01,
    "MIN_BATCH_SIZE": 8,
    "MAX_BATCH_SIZE": 1024,
    "MIN_LEARN_STEP": 256,
    "MAX_LEARN_STEP": 8192,
}

# Create the Environment

In [3]:
num_envs=8
env = make_vect_envs("CartPole-v1", num_envs=num_envs)  # Create environment
try:
    state_dim = env.single_observation_space.n  # Discrete observation space
    one_hot = True  # Requires one-hot encoding
except Exception:
    state_dim = env.single_observation_space.shape  # Continuous observation space
    one_hot = False  # Does not require one-hot encoding
try:
    action_dim = env.single_action_space.n  # Discrete action space
except Exception:
    action_dim = env.single_action_space.shape[0]  # Continuous action space

if INIT_HP["CHANNELS_LAST"]:
    # Adjust dimensions for PyTorch API (C, H, W), for envs with RGB image states
    state_dim = (state_dim[2], state_dim[0], state_dim[1])

# Create a Population of Agents

In [4]:
# Set-up the device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Define the network configuration of a simple mlp with two hidden layers, each with 64 nodes
net_config = {"arch": "mlp", "hidden_size": [64, 64]}

# Define a population
pop = create_population(
    algo="PPO",  # Algorithm
    state_dim=state_dim,  # State dimension
    action_dim=action_dim,  # Action dimension
    one_hot=one_hot,  # One-hot encoding
    net_config=net_config,  # Network configuration
    INIT_HP=INIT_HP,  # Initial hyperparameter
    population_size=INIT_HP["POP_SIZE"],  # Population size
    num_envs=num_envs,
    device=device,
)

# Creating Mutations and Tournament objects

In [5]:
tournament = TournamentSelection(
    INIT_HP["TOURN_SIZE"],
    INIT_HP["ELITISM"],
    INIT_HP["POP_SIZE"],
    INIT_HP["EVAL_LOOP"],
)

In [6]:
mutations = Mutations(
    algo="PPO",
    no_mutation=MUT_P["NO_MUT"],
    architecture=MUT_P["ARCH_MUT"],
    new_layer_prob=MUT_P["NEW_LAYER"],
    parameters=MUT_P["PARAMS_MUT"],
    activation=MUT_P["ACT_MUT"],
    rl_hp=MUT_P["RL_HP_MUT"],
    rl_hp_selection=MUT_P["RL_HP_SELECTION"],
    min_lr=MUT_P["MIN_LR"],
    max_lr=MUT_P["MAX_LR"],
    min_batch_size=MUT_P["MAX_BATCH_SIZE"],
    max_batch_size=MUT_P["MAX_BATCH_SIZE"],
    min_learn_step=MUT_P["MIN_LEARN_STEP"],
    max_learn_step=MUT_P["MAX_LEARN_STEP"],
    mutation_sd=MUT_P["MUT_SD"],
    arch=net_config["arch"],
    rand_seed=MUT_P["RAND_SEED"],
    device=device,
)

# Training and Saving an Agent

In [7]:
# Define a save path for our trained agent
save_path = "PPO_cartpole_trained_agent.pt"

trained_pop, pop_fitnesses = train_on_policy(
    env=env,
    env_name="Acrobot-v1",
    algo="PPO",
    pop=pop,
    INIT_HP=INIT_HP,
    MUT_P=MUT_P,
    swap_channels=INIT_HP["CHANNELS_LAST"],
    max_steps=INIT_HP["MAX_STEPS"],
    evo_steps=INIT_HP["EVO_STEPS"],
    eval_steps=INIT_HP["EVAL_STEPS"],
    eval_loop=INIT_HP["EVAL_LOOP"],
    tournament=tournament,
    mutation=mutations,
    wb=False,  # Boolean flag to record run with Weights & Biases
    save_elite=True,  # Boolean flag to save the elite agent in the population
    elite_path=save_path,
)


Training...


  7%|6         | 10496/150000 [  01:57<  35:47, 64.97step/s] 


                --- Global Steps 40960 ---
                Fitness:		['277.00', '263.92', '364.33', '417.00']
                Score:		[58.21656050955414, 68.65413533834587, 73.79508196721312, 72.10483870967742]
                5 fitness avgs:	['417.00', '277.00', '277.00', '417.00']
                10 score avgs:	['218.10', '232.30', '232.30', '218.10']
                Agents:		[3, 4, 5, 6]
                Steps:		[10240, 10240, 10240, 10240]
                Mutations:		['None', 'bs', 'param', 'None']


 14%|#3        | 20736/150000 [  03:37<  40:40, 52.97step/s] 


                --- Global Steps 81920 ---
                Fitness:		['247.75', '401.50', '491.00', '259.79']
                Score:		[290.39285714285717, 332.64, 354.96, 319.0689655172414]
                5 fitness avgs:	['384.00', '338.40', '338.40', '384.00']
                10 score avgs:	['375.70', '215.30', '215.30', '375.70']
                Agents:		[5, 7, 8, 9]
                Steps:		[20480, 20480, 20480, 20480]
                Mutations:		['None', 'param', 'param', 'arch']


...


                --- Global Steps 532480 ---
                Fitness:		['459.29', '434.67', '449.25', '500.00']
                Score:		[400.65, 414.05, 500.0, 456.05555555555554]
                5 fitness avgs:	['499.71', '499.71', '474.70', '491.57']
                10 score avgs:	['475.20', '475.20', '424.30', '469.20']
                Agents:		[39, 40, 41, 42]
                Steps:		[133120, 133120, 133120, 133120]
                Mutations:		['bs', 'lr', 'None', 'None']


 96%|#########5| 143616/150000 [  16:17<  01:13, 86.60step/s] 


                --- Global Steps 573440 ---
                Fitness:		['500.00', '500.00', '500.00', '386.54']
                Score:		[476.11764705882354, 500.0, 470.6111111111111, 371.4782608695652]
                5 fitness avgs:	['500.00', '500.00', '500.00', '500.00']
                10 score avgs:	['459.40', '459.40', '500.00', '500.00']
                Agents:		[39, 43, 44, 45]
                Steps:		[143360, 143360, 143360, 143360]
                Mutations:		['None', 'None', 'None', 'None']


|          | 153600/? [  17:08<  00:00, 149.27step/s]         


                --- Global Steps 614400 ---
                Fitness:		['356.08', '447.54', '483.54', '500.00']
                Score:		[495.9375, 491.5625, 435.8421052631579, 497.375]
                5 fitness avgs:	['500.00', '489.51', '489.51', '500.00']
                10 score avgs:	['495.80', '486.50', '486.50', '495.80']
                Agents:		[45, 46, 47, 48]
                Steps:		[153600, 153600, 153600, 153600]
                Mutations:		['None', 'None', 'None', 'None']


In [8]:
total_steps = 0

# TRAINING LOOP
print("Training...")
pbar = trange(INIT_HP["MAX_STEPS"], unit="step")
while np.less([agent.steps[-1] for agent in pop], INIT_HP["MAX_STEPS"]).all():
    pop_episode_scores = []
    for agent in pop:  # Loop through population
        state, info = env.reset()  # Reset environment at start of episode
        scores = np.zeros(num_envs)
        completed_episode_scores = []
        steps = 0

        for _ in range(-(INIT_HP["EVO_STEPS"] // -agent.learn_step)):

            states = []
            actions = []
            log_probs = []
            rewards = []
            dones = []
            values = []

            learn_steps = 0

            for idx_step in range(-(agent.learn_step // -num_envs)):
                if INIT_HP["CHANNELS_LAST"]:
                    state = np.moveaxis(state, [-1], [-3])

                # Get next action from agent
                action, log_prob, _, value = agent.get_action(state)

                # Act in environment
                next_state, reward, terminated, truncated, info = env.step(action)

                total_steps += num_envs
                steps += num_envs
                learn_steps += num_envs

                states.append(state)
                actions.append(action)
                log_probs.append(log_prob)
                rewards.append(reward)
                dones.append(terminated)
                values.append(value)

                state = next_state
                scores += np.array(reward)

                for idx, (d, t) in enumerate(zip(terminated, truncated)):
                    if d or t:
                        completed_episode_scores.append(scores[idx])
                        agent.scores.append(scores[idx])
                        scores[idx] = 0

            pbar.update(learn_steps // len(pop))

            if INIT_HP["CHANNELS_LAST"]:
                next_state = np.moveaxis(next_state, [-1], [-3])

            experiences = (
                states,
                actions,
                log_probs,
                rewards,
                dones,
                values,
                next_state,
            )
            # Learn according to agent's RL algorithm
            agent.learn(experiences)

        agent.steps[-1] += steps
        pop_episode_scores.append(completed_episode_scores)

    # Evaluate population
    fitnesses = [
        agent.test(
            env,
            swap_channels=INIT_HP["CHANNELS_LAST"],
            max_steps=INIT_HP["EVAL_STEPS"],
            loop=INIT_HP["EVAL_LOOP"],
        )
        for agent in pop
    ]
    mean_scores = [
        (
            np.mean(episode_scores)
            if len(episode_scores) > 0
            else "0 completed episodes"
        )
        for episode_scores in pop_episode_scores
    ]

    print(f"--- Global steps {total_steps} ---")
    print(f"Steps {[agent.steps[-1] for agent in pop]}")
    print(f"Scores: {mean_scores}")
    print(f'Fitnesses: {["%.2f"%fitness for fitness in fitnesses]}')
    print(
        f'5 fitness avgs: {["%.2f"%np.mean(agent.fitness[-5:]) for agent in pop]}'
    )

    # Tournament selection and population mutation
    elite, pop = tournament.select(pop)
    pop = mutations.mutation(pop)

    # Update step counter
    for agent in pop:
        agent.steps.append(agent.steps[-1])

# Save the trained algorithm
elite.save_checkpoint(save_path)

pbar.close()
env.close()

Training...


  7%|███████████▍                                                                                                                                                        | 10496/150000 [01:46<34:25, 67.54step/s]

--- Global steps 40960 ---
Steps [20480, 20480, 20480, 20480]
Scores: [251.9142857142857, 253.6969696969697, 334.16, 207.7906976744186]
Fitnesses: ['359.67', '311.88', '434.38', '218.00']
5 fitness avgs: ['318.33', '287.90', '399.35', '317.50']


 14%|██████████████████████▋                                                                                                                                             | 20736/150000 [03:37<34:32, 62.37step/s]

--- Global steps 81920 ---
Steps [30720, 30720, 30720, 30720]
Scores: [346.1111111111111, 247.05882352941177, 412.95238095238096, 415.15]
Fitnesses: ['238.71', '339.29', '387.62', '384.92']
5 fitness avgs: ['345.81', '324.76', '395.44', '394.54']


...

 89%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                  | 133120/150000 [30:31<02:46, 101.64step/s]

--- Global steps 532480 ---
Steps [143360, 143360, 143360, 143360]
Scores: [347.2692307692308, 435.94444444444446, 412.76190476190476, 500.0]
Fitnesses: ['433.88', '438.25', '451.12', '495.88']
5 fitness avgs: ['467.69', '456.93', '464.87', '480.09']


 96%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊       | 143360/150000 [33:16<02:22, 46.59step/s]

--- Global steps 573440 ---
Steps [153600, 153600, 153600, 153600]
Scores: [471.5, 462.55555555555554, 472.5882352941176, 480.11764705882354]
Fitnesses: ['394.42', '500.00', '500.00', '498.25']
5 fitness avgs: ['459.95', '468.67', '481.07', '468.32']


 96%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊       | 143360/150000 [33:26<01:32, 71.44step/s]
